In [1]:
import cml.data_v1 as cmldata

CONNECTION_NAME = "go01-aw-dl"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

# Sample usage to run query through spark
EXAMPLE_SQL_QUERY = "show databases"
spark.sql(EXAMPLE_SQL_QUERY).show()

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco
Hive Session ID = b50b5da0-a4f5-426c-b06d-15ade7b718db


+--------------------+
|           namespace|
+--------------------+
|         01_car_data|
|           01_car_dw|
|      adash_car_data|
|             airline|
|          airline_dw|
|            airlines|
|        airlines_csv|
|       airlines_csv1|
|   airlines_csv_vish|
|    airlines_iceberg|
|   airlines_iceberg1|
|airlines_iceberg_...|
|      airlines_mjain|
|          airquality|
|          atlas_demo|
|            bankdemo|
|              bhagan|
|             cdedemo|
|        cdp_overview|
|        cgsifacebook|
+--------------------+
only showing top 20 rows



In [2]:
spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.repl.local.jars',
  'file:///runtime-addons/spark320-17-hf1-6xa3lk/opt/spark/optional-lib/iceberg-spark-runtime-3.2_2.12-0.14.1.1.17.7215.0-31.jar'),
 ('spark.network.crypto.enabled', 'true'),
 ('spark.ui.proxyRedirectUri',
  'https://spark-6byd321my797fh7l.ml-4c5feac0-3ec.go01-dem.ylcu-atmi.cloudera.site'),
 ('spark.sql.hive.hwc.execution.mode', 'spark'),
 ('spark.driver.memory', '1525m'),
 ('spark.kerberos.renewal.credentials', 'ccache'),
 ('spark.sql.catalog.spark_catalog',
  'org.apache.iceberg.spark.SparkSessionCatalog'),
 ('spark.dynamicAllocation.maxExecutors', '49'),
 ('spark.eventLog.dir', 'file:///sparkeventlogs'),
 ('spark.hadoop.yarn.resourcemanager.principal', 'pauldefusco'),
 ('spark.kubernetes.driver.annotation.cluster-autoscaler.kubernetes.io/safe-to-evict',
  'false'),
 ('spark.ui.port', '20049'),
 ('spark.driver.port', '39115'),
 ('spark.yarn.access.hadoopFileSystems',
  's3a://go01-demo/warehouse/tablespace/external/hive'

In [3]:
# Show catalog and database
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|  default|
+-------------+---------+



In [4]:
# Create a new database
spark.sql("CREATE DATABASE IF NOT EXISTS spark_catalog.lakehouse")
spark.sql("USE spark_catalog.lakehouse")

DataFrame[]

In [5]:
spark.sql("DROP TABLE IF EXISTS test_table")

DataFrame[]

In [6]:
spark.sql("CREATE TABLE IF NOT EXISTS test_table\
            (id BIGINT, state STRING, country STRING)\
            USING iceberg")

DataFrame[]

In [7]:
spark.sql("INSERT INTO lakehouse.test_table VALUES (1, 'CA', 'USA'),(2, 'CA', 'USA'),\
                    (3, 'AZ', 'USA'),\
                    (4, 'ON', 'CAN'),\
                    (5, 'AL', 'CAN')")

DataFrame[]

In [8]:
spark.sql("SELECT * FROM lakehouse.test_table.partitions").show()

+------------+----------+
|record_count|file_count|
+------------+----------+
|           5|         2|
+------------+----------+



In [31]:
# SHOW PARTITIONS not supported by Iceberg
#spark.sql("SHOW PARTITIONS lakehouse.test_table")

In [19]:
metadata_path = "s3a://go01-demo/warehouse/tablespace/external/hive/lakehouse.db/test_table/metadata"

In [11]:
import boto3

s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

metadata_file_list = []

for object_summary in my_bucket.objects.filter(Prefix="warehouse/tablespace/external/hive/lakehouse.db/test_table/metadata"):
    print(object_summary.key)
    metadata_file_list.append(object_summary.key)

warehouse/tablespace/external/hive/lakehouse.db/test_table/metadata/00000-45adadbe-3f75-47c7-9def-3ae4d78ef772.metadata.json
warehouse/tablespace/external/hive/lakehouse.db/test_table/metadata/00001-cacdd227-cc50-4629-8c66-22ccc741c645.metadata.json
warehouse/tablespace/external/hive/lakehouse.db/test_table/metadata/6400b626-76b5-4dce-beeb-56661d8156b2-m0.avro
warehouse/tablespace/external/hive/lakehouse.db/test_table/metadata/snap-5349527462802761390-1-6400b626-76b5-4dce-beeb-56661d8156b2.avro


In [12]:
import pandas as pd
spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[1]).toPandas()

,current-schema-id,current-snapshot-id,default-sort-order-id,default-spec-id,format-version,last-column-id,last-partition-id,last-updated-ms,location,metadata-log,partition-spec,partition-specs,properties,refs,schema,schemas,snapshot-log,snapshots,sort-orders,table-uuid
0,0,5349527462802761390,0,0,1,3,999,1682722064311,s3a://go01-demo/warehouse/tablespace/external/...,[(s3a://go01-demo/warehouse/tablespace/externa...,[],"[([], 0)]","(pauldefusco,)","((5349527462802761390, branch),)","([(1, id, False, long), (2, state, False, stri...","[([Row(id=1, name='id', required=False, type='...","[(5349527462802761390, 1682722064311)]",[(s3a://go01-demo/warehouse/tablespace/externa...,"[([], 0)]",a08ebae6-5aeb-45c0-b52a-fd74ba6eea9b


In [13]:
print("Showing " + metadata_file_list[3])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[3]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/test_table/metadata/snap-5349527462802761390-1-6400b626-76b5-4dce-beeb-56661d8156b2.avro


,manifest_path,manifest_length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,partitions,added_rows_count,existing_rows_count,deleted_rows_count
0,s3a://go01-demo/warehouse/tablespace/external/...,5963,0,5349527462802761390,2,0,0,[],5,0,0


In [14]:
print("Showing " + metadata_file_list[3])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[2]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse.db/test_table/metadata/snap-5349527462802761390-1-6400b626-76b5-4dce-beeb-56661d8156b2.avro


,status,snapshot_id,data_file
0,1,5349527462802761390,(s3a://go01-demo/warehouse/tablespace/external...
1,1,5349527462802761390,(s3a://go01-demo/warehouse/tablespace/external...


In [7]:
from pyspark.sql.functions import date_format